## Описание спайдера

Был создан спайдер, собирающий страницы с вики ([код спайдера](https://github.com/ikrivulets/PageRank/tree/master/wiki_spider)).
Он ничинал с двух стартовых страниц: 
~~~~ python
start_urls = ['https://en.wikipedia.org/wiki/Los_Angeles',
              'https://en.wikipedia.org/wiki/Information_retrieval']
~~~~

Настройки спайдера:

~~~~ python
# кол-во параллельно обрабатываемых запросов
CONCURRENT_REQUESTS = 32
# задержка перед запросом
DOWNLOAD_DELAY = 0.1
# условие остановки, количество айтемов
CLOSESPIDER_ITEMCOUNT = 10000
~~~~

Айтемы записывались в json lines (.jl) файл в след формате:

~~~~ json
{
    "id": "...",
    "url": "...",
    "snippet": "...",
    "links": ["...", "..."],
}
~~~~

Он проработал около 4.5 часов и собрал **100034** айтема. Для это использовалась плтформа [scrapinghub](http://scrapinghub.com "scrapinghub")
Но он собрал айтемы не только английско википедии. Для начала посмотрим только английскую часть ([en.wikipedia.org](en.wikipedia.org)). А в конце сравним с общими результатами.  
Английских статей - **42092**

## Парсим файл и создаем граф:

In [1]:
import json
import networkx as nx

items_file = 'items_full.jl'
item_ids = {}
item_ids_inv = {}
item_relations = {}

def is_english_wiki(url):
    return 'en.wiki' in url

with open(items_file) as item_json:
    for line in item_json:
        item = json.loads(line)
        if is_english_wiki(item['url']):
            item_ids[item['url']] = item['id']
            item_ids_inv[item['id']] = item['url']

with open(items_file) as item_json:
    for line in item_json:
        item = json.loads(line)
        if is_english_wiki(item['url']):
            item_relations[item['id']] = [item_ids[url] for url in item['links'] if url in item_ids]

graph = nx.DiGraph()
graph.add_nodes_from(item_relations)
for from_id in item_relations:
    graph.add_edges_from([(from_id, to_id) for to_id in item_relations[from_id]])

### Взглянем, на распределение степеней вершин графа:

In [5]:
degrees = list(graph.degree(graph.nodes()).values())
len(degrees)

42092

In [30]:
import matplotlib.pyplot as plt
%matplotlib inline

degrees = sorted(degrees)
from itertools import groupby
groups = [(key, len(list(group))) for key, group in groupby(degrees)]
# x = [el[0] for el in groups]
# y = [el[1] for el in groups]
# plt.plot(x, y)
groups[:40]

[(0, 191),
 (1, 4135),
 (2, 7631),
 (3, 8915),
 (4, 7522),
 (5, 5173),
 (6, 3329),
 (7, 1948),
 (8, 1142),
 (9, 657),
 (10, 345),
 (11, 224),
 (12, 179),
 (13, 124),
 (14, 80),
 (15, 60),
 (16, 41),
 (17, 44),
 (18, 34),
 (19, 25),
 (20, 18),
 (21, 24),
 (22, 20),
 (23, 14),
 (24, 18),
 (25, 14),
 (26, 10),
 (27, 10),
 (28, 8),
 (29, 14),
 (30, 5),
 (31, 6),
 (32, 5),
 (33, 8),
 (34, 6),
 (35, 7),
 (36, 3),
 (37, 1),
 (38, 7),
 (39, 6)]

### Не думаю, что график может изобразить распределение степеней вершин понятнее, чем этот список, где видим, что вершин, со степенью до 10 очень много, тогда как с большими степенями их число потом резко убывает. 

### Реалиация алгоритма PageRank. Основная идея в том, что сначала инициализируется одинаковые веса для всех узлов, а затем итеративно с заданным весом alpha обновляем узлы графа по формуле $Rank(x) = \frac{1-\lambda}{N} + \lambda \sum\limits_{y->x} \frac{Rank(y)}{out(y)}$, где y - вершины, которые имеют входящие в x дуги, а out(y) - количества выходящих из y дуг:

In [34]:
class PageRank:
    def __init__(self, graph, alpha=0.85):
        self.graph = graph
        self.V = len(self.graph)
        self.d = alpha
        self.ranks = dict()
    
    def rank(self):
        for key in self.graph.nodes():
            self.ranks[key] = 1 / float(self.V)

        for _ in range(10):
            for key, node in self.graph.nodes(data=True):
                rank_sum = 0
                neighbors = self.graph.in_edges(key)
                for n in neighbors:
                    outlinks = len(self.graph.out_edges(n[0]))
                    if outlinks > 0:
                        rank_sum += (1 / float(outlinks)) * self.ranks[n[0]]
                # actual page rank compution
                self.ranks[key] = ((1 - float(self.d)) * (1/float(self.V))) + self.d*rank_sum

        return self
p = PageRank(graph)
p.rank()

<__main__.PageRank instance at 0x7f123b84af38>

### Получили топ10 английских статей с наибольшим ранком:

In [35]:
import operator
sorted_r = sorted(p.ranks.iteritems(), key=operator.itemgetter(1), reverse=True)
[item_ids_inv[tup[0]] for tup in sorted_r][:10]

[u'https://en.wikipedia.org/wiki/World_War_II',
 u'https://en.wikipedia.org/wiki/Pacific_Ocean',
 u'https://en.wikipedia.org/wiki/London',
 u'https://en.wikipedia.org/wiki/Bolshevik',
 u'https://en.wikipedia.org/wiki/Barack_Obama',
 u'https://en.wikipedia.org/wiki/Spanish_language',
 u'https://en.wikipedia.org/wiki/Pinko',
 u'https://en.wikipedia.org/wiki/Cantar_de_mio_Cid',
 u'https://en.wikipedia.org/wiki/Portugal',
 u'https://en.wikipedia.org/wiki/Monsanto']

## Pagerank с дефолтным значением alpha(реализация networkx для сравнения)

In [33]:
def print_top_info(top10, pagerank, show_snippet=False):
    top10urls = {}
    with open(items_file) as item_json:
        for i, line in enumerate(item_json):
            if i in top10:
                item = json.loads(line)
                top10urls[i] = item
    for i in top10:
        print top10urls[i]['url']
        print 'Pagerank: ', pagerank[i]
        if show_snippet:
            print top10urls[i]['snippet']
        print

pagerank = nx.pagerank(graph)
top10 = sorted(pagerank, key=lambda k: pagerank[k], reverse=True)[:10]
print_top_info(top10, pagerank, True)

https://en.wikipedia.org/wiki/World_War_II
Pagerank:  0.0147880292089
World War II (often abbreviated to WWII or WW2), also known as the Second World War, was a global war that lasted from 1939 to 1945, although related conflicts began earlier. It involved the vast majority of the world's countries—including all of the grea...

https://en.wikipedia.org/wiki/Pacific_Ocean
Pagerank:  0.0116592539118
The Pacific Ocean is the largest and deepest of the Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean (or, depending on definition, to Antarctica) in the south and is bounded by Asia and Australia in the west a...

https://en.wikipedia.org/wiki/London
Pagerank:  0.00700272726047
London i/ˈlʌndən/ is the capital and most populous city of England and the United Kingdom.[7][8] Standing on the River Thames in the south east of the island of Great Britain, London has been a major settlement for two millennia. It was founded by the Rom...

https://en.wik


**Дальше будем выводить без сниппета, потому что он занимает много места.  
Приведем ниже топ с другими значениями alpha:**
### Как видим, результаты почти те же, за исключением перестановок
## alpha = 0.3

In [38]:
p = PageRank(graph, alpha=0.3)
p.rank()
sorted_r = sorted(p.ranks.iteritems(), key=operator.itemgetter(1), reverse=True)
[item_ids_inv[tup[0]] for tup in sorted_r][:10]

[u'https://en.wikipedia.org/wiki/World_War_II',
 u'https://en.wikipedia.org/wiki/London',
 u'https://en.wikipedia.org/wiki/Pacific_Ocean',
 u'https://en.wikipedia.org/wiki/Los_Angeles',
 u'https://en.wikipedia.org/wiki/Spanish_language',
 u'https://en.wikipedia.org/wiki/Bolshevik',
 u'https://en.wikipedia.org/wiki/Portugal',
 u'https://en.wikipedia.org/wiki/Barack_Obama',
 u'https://en.wikipedia.org/wiki/Roman_Catholic',
 u'https://en.wikipedia.org/wiki/South_America']

In [36]:
pagerank = nx.pagerank(graph, 0.3)
top10 = sorted(pagerank, key=lambda k: pagerank[k], reverse=True)[:10]
print_top_info(top10, pagerank)

https://en.wikipedia.org/wiki/World_War_II
Pagerank:  0.00558947774906

https://en.wikipedia.org/wiki/London
Pagerank:  0.0026815674597

https://en.wikipedia.org/wiki/Los_Angeles
Pagerank:  0.00130647888071

https://en.wikipedia.org/wiki/Pacific_Ocean
Pagerank:  0.00130075019907

https://en.wikipedia.org/wiki/Spanish_language
Pagerank:  0.00116745750868

https://en.wikipedia.org/wiki/Bolshevik
Pagerank:  0.00090263100118

https://en.wikipedia.org/wiki/Portugal
Pagerank:  0.000845448870245

https://en.wikipedia.org/wiki/Barack_Obama
Pagerank:  0.000751013302733

https://en.wikipedia.org/wiki/Roman_Catholic
Pagerank:  0.000669814271271

https://en.wikipedia.org/wiki/South_America
Pagerank:  0.000409215329487



### Снова почти идентичны

Как мы видим,на первом месте всегда https://en.wikipedia.org/wiki/World_War_II.  
Дальше места меняются в зависимости от alpha. Сам список топ10 - меняется не сильно (+-1 элемент).

Теперь посмотрим результат для все ссылок вместо, когда их 100034:

In [41]:
items_file = 'items_full.jl'
item_ids_full = {}
item_ids_inv = {}
item_relations_full = {}


with open(items_file) as item_json:
    for line in item_json:
        item = json.loads(line)
        item_ids_full[item['url']] = item['id']
        item_ids_inv[item['id']] = item['url']
        

with open(items_file) as item_json:
    for line in item_json:
        item = json.loads(line)
        item_relations_full[item['id']] = [item_ids_full[url] for url in item['links'] if url in item_ids_full]

graph_full = nx.DiGraph()
graph_full.add_nodes_from(item_relations_full)
for from_id in item_relations_full:
    graph_full.add_edges_from([(from_id, to_id) for to_id in item_relations_full[from_id]])

## alphа - 0.85

### Топ моей реализации(в топ попадают русские статьи про немецкий язык и Германию):

In [42]:
p = PageRank(graph_full)
p.rank()
sorted_r = sorted(p.ranks.iteritems(), key=operator.itemgetter(1), reverse=True)
[item_ids_inv[tup[0]] for tup in sorted_r][:10]

[u'https://en.wikipedia.org/wiki/World_War_II',
 u'https://en.wikipedia.org/wiki/Pacific_Ocean',
 u'https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%BC%D0%B5%D1%86%D0%BA%D0%B8%D0%B9_%D1%8F%D0%B7%D1%8B%D0%BA',
 u'https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D1%80%D0%BC%D0%B0%D0%BD%D0%B8%D1%8F',
 u'https://en.wikipedia.org/wiki/London',
 u'https://en.wikipedia.org/wiki/Bolshevik',
 u'https://en.wikipedia.org/wiki/Barack_Obama',
 u'https://en.wikipedia.org/wiki/Spanish_language',
 u'https://en.wikipedia.org/wiki/Pinko',
 u'https://it.wikipedia.org/wiki/1929']

### Топ NetworkX PageRank

In [196]:
pagerank_full = nx.pagerank(graph_full)
top10_full = sorted(pagerank_full, key=lambda k: pagerank_full[k], reverse=True)[:10]
print_top_info(top10_full, pagerank_full, True)

https://en.wikipedia.org/wiki/World_War_II
Pagerank:  0.00654265735122
World War II (often abbreviated to WWII or WW2), also known as the Second World War, was a global war that lasted from 1939 to 1945, although related conflicts began earlier. It involved the vast majority of the world's countries—including all of the grea...

https://en.wikipedia.org/wiki/Pacific_Ocean
Pagerank:  0.00448297229308
The Pacific Ocean is the largest and deepest of the Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean (or, depending on definition, to Antarctica) in the south and is bounded by Asia and Australia in the west a...

https://en.wikipedia.org/wiki/London
Pagerank:  0.00309974345443
London i/ˈlʌndən/ is the capital and most populous city of England and the United Kingdom.[7][8] Standing on the River Thames in the south east of the island of Great Britain, London has been a major settlement for two millennia. It was founded by the Rom...

https://en.w

И в этом случае топ документов не сильно поменялся. 

Как мы видим при разных alpha в топ пробиваются итальянские, росскийские и немецкие статьи.  
